# TD3 — Choix d'Architecture OLAP (ROLAP vs MOLAP vs HOLAP)

**Durée estimée** : 1h30  
**Niveau** : Avancé  
**Objectifs pédagogiques** :
1.  **Analyser** des contraintes métier (Volumétrie, Latence, Coût).
2.  **Comparer** les architectures techniques (Relationnel vs Multidimensionnel).
3.  **Simuler** un benchmark de performance.
4.  **Concevoir** une architecture Hybride (HOLAP) et un plan d'actualisation.

---

In [ ]:
## Partie 1 : Analyse du Contexte

### Le Scénario "Retail Big Data"
-   **Volumétrie** : 50 Millions de lignes / an. Historique sur 5 ans (250M lignes).
-   **Usage** :
    -   **Tableaux de bord (Dashboards)** : Très consultés, focus sur les 60 derniers jours. SLA < 5 sec.
    -   **Analyses Ad-hoc** : Exploratoires, sur tout l'historique. SLA < 20 sec.
-   **Contrainte** : Budget stockage limité (Le stockage MOLAP coûte cher).

### Le Dilemme
-   Tout mettre en **ROLAP** ? Risque de lenteur sur les Dashboards.
-   Tout mettre en **MOLAP** ? Explosion des coûts de stockage et temps de calcul du cube.
-   La solution : **HOLAP (Hybrid)**.

---

## Partie 2 : Comparatif Technique (Benchmark Simulé)

Nous allons utiliser Python pour visualiser les compromis entre les architectures.

### Critères de comparaison :
1.  **Latence** : Temps de réponse d'une requête.
2.  **Coût Stockage** : Volume disque consommé (Index vs Cubes).
3.  **Fraîcheur** : Temps de mise à jour (ETL vs Processing Cube).

import pandas as pd
import matplotlib.pyplot as plt

# Données simulées pour le benchmark
data = {
    'Architecture': ['ROLAP', 'MOLAP', 'HOLAP'],
    'Latence Moyenne (s)': [18.5, 3.2, 6.5],
    'Coût Stockage (Index 100)': [100, 400, 150], # MOLAP explose souvent x4 à x10 selon la densité
    'Flexibilité (Ad-hoc)': ['Haute', 'Faible', 'Moyenne'],
    'Complexité ETL': ['Faible', 'Moyenne', 'Élevée']
}

df = pd.DataFrame(data)

# Affichage du tableau comparatif
print("📊 Tableau Comparatif des Architectures :")
display(df)

# Visualisation (Optionnel si matplotlib dispo)
# df.set_index('Architecture')[['Latence Moyenne (s)', 'Coût Stockage (Index 100)']].plot(kind='bar', secondary_y='Coût Stockage (Index 100)')

In [ ]:
## Partie 3 : La Solution Préconisée (HOLAP)

### Pourquoi HOLAP ?
C'est le compromis "Meilleur des deux mondes" pour ce cas d'usage :
1.  **Données Récentes (Hot Data)** : Stockées en **MOLAP** (Cube).
    -   *Périmètre* : 60 derniers jours.
    -   *Gain* : Réponse instantanée pour les tableaux de bord quotidiens.
2.  **Historique (Cold Data)** : Laissé en **ROLAP** (Base Relationnelle).
    -   *Périmètre* : 5 ans.
    -   *Gain* : Coût de stockage faible, accessible pour les requêtes ad-hoc ponctuelles.

### Schéma d'Architecture Cible
Le flux de données doit alimenter deux cibles.

```mermaid
flowchart TD
  Sources[Systèmes Sources (ERP, CRM)] -->|Extract| Staging[Zone de Staging]
  
  subgraph "Entrepôt de Données (DWH)"
    Staging -->|ETL| ROLAP[Base ROLAP (Détail Historique)]
  end
  
  subgraph "Accélération (Performance)"
    ROLAP -->|Processing| MOLAP[Cube MOLAP (60 jours glissants)]
  end
  
  subgraph "Restitution"
    MOLAP -->|Requêtes Rapides| Dashboards[Tableaux de Bord]
    ROLAP -->|Requêtes Lourdes| AdHoc[Analyses Ad-Hoc / Data Science]
  end
  
  style MOLAP fill:#ff9,stroke:#333
  style ROLAP fill:#9cf,stroke:#333
```

## Partie 4 : Plan d'Actualisation et SLA

Pour garantir la performance et la fraîcheur, nous définissons des règles précises (SLA - Service Level Agreement).

### 1. Stratégie de Partitionnement
Pour éviter de tout recalculer chaque nuit :
-   **Partitionnement Temporel** sur la table de faits.
-   Chaque jour, on ne charge que la partition J-1.

### 2. Fréquence de mise à jour
| Cible | Fréquence | Méthode | Fenêtre de tir |
|-------|-----------|---------|----------------|
| **ROLAP** | Quotidienne (J+1) | Insert Incrémental | 02:00 - 04:00 |
| **MOLAP** | Quotidienne (J+1) | Process Partition | 04:00 - 05:00 |
| **Historique** | Mensuelle | Archivage / Compression | Week-end |

### 3. Simulation SQL (Partitionnement)
Voici comment on implémente le partitionnement (conceptuel) en SQL pour faciliter la gestion HOLAP.

import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Simulation : Table partitionnée (en SQL standard, on utilise PARTITION BY)
# Ici on simule avec une colonne 'partition_key'
cursor.execute('''
CREATE TABLE fact_ventes_partitioned (
    vente_id INTEGER PRIMARY KEY,
    date_vente DATE,
    montant REAL,
    partition_key TEXT -- ex: '2024-01', '2024-02'
);
''')

# Insertion de données dans des partitions différentes
cursor.executescript('''
INSERT INTO fact_ventes_partitioned VALUES (1, '2024-01-15', 100, 'COLD_2024_01');
INSERT INTO fact_ventes_partitioned VALUES (2, '2024-02-20', 200, 'HOT_2024_02'); -- Récent -> Vers MOLAP
''')

print("✅ Table partitionnée simulée.")

# Requête de maintenance : Identifier les données à charger en MOLAP (Hot)
cursor.execute("SELECT count(*) FROM fact_ventes_partitioned WHERE partition_key LIKE 'HOT%'")
nb_hot = cursor.fetchone()[0]
print(f"🔥 Données à charger en MOLAP : {nb_hot} lignes")

conn.close()

## Conclusion et Risques

### Risques identifiés :
1.  **Désynchronisation** : Si le chargement MOLAP échoue, les dashboards affichent des données différentes du ROLAP.
    *   *Parade* : Monitoring strict et alerte si écart > 1%.
2.  **Explosion du Cube** : Si on étend la fenêtre de 60 jours à 1 an sans contrôle.
    *   *Parade* : Purge automatique des partitions anciennes dans le cube.
3.  **Complexité** : Maintenir deux systèmes (ETL ROLAP + Cube MOLAP).

### Synthèse
L'architecture **HOLAP** est la réponse standard aux problématiques Big Data modernes, permettant de concilier volume historique et performance immédiate.

---
**Fin du TD3**